[View in Colaboratory](https://colab.research.google.com/github/mitreilab/autonomousfuture/blob/master/reilAutomation.ipynb)

## Import OES Data

In [0]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as pp
!wget "https://www.dropbox.com/s/nb634owp9471quf/rawdatanaics.csv?dl=0" -q -O "rawdatanaics.csv" 
#Download raw csv file from Dropbox and rename to "rawdatanaics.csv"
oes = pd.read_csv("rawdatanaics.csv",low_memory=False)



## Import ONet Data ##

In [0]:
!wget "https://www.dropbox.com/s/2y7c9yo93nyq8uh/combined.sqlite?dl=0" -q -O  "onet.sqlite"  
#Download raw SQLite file from Dropbox and rename to "onet.sqlite"
import sqlite3
onetConnection = sqlite3.connect('onet.sqlite')
cur = onetConnection.cursor()
query = "SELECT * FROM occupation_data";
occdata = pandas.read_sql_query(query, onetConnection)
len(occdata)
# Onet surveyed 1109 occupations.


## Data cleaning

### Target the occupations that are 1. In Real Estate industry 2. Not General Group 3. Onet has surveyed


In [21]:
!wget "https://www.dropbox.com/s/329edv8yrn3ik25/Occupation%20Data.csv?dl=0"-q -O  "Occupation Data.csv" 
occ_onet=pd.read_csv("Occupation Data.csv",low_memory=False)
#Download 'Occupation Data' CSV that changes the 8-digit Onet-code to 6-digit, ommiting the last two digits.
del_occ_endwith_00_and_in_Onet=oes.loc[oes['occ_code'].isin(occ_onet['O*NET-SOC Code'])]
#Delete the occupation if the code is not in 'Occupation Data'
del_occ_endwith_00_and_in_Onet.shape


Redirecting output to ‘wget-log.15’.


(22026, 61)

In [26]:
target_occ=set(del_occ_endwith_00_and_in_Onet['occ_code'])
target_occ=list(target_occ)
target=pd.DataFrame({'target_occ': target_occ})
len(target)
# we target 558 occupations.

558

### Drop the columns in the oes.csv that contains duplicated or redundant data.

In [27]:
del_occ_endwith_00_and_in_Onet.columns
#drop the colomuns that contain duplicate information
del_duplicated_column=del_occ_endwith_00_and_in_Onet.drop(['title_code','occ_title_code', 'occ_title_mid', 'year.1', 'naics.1', 'naics_title.1', 'occ_code.1', 'occ_title.1', 'group', 'naics_codedescription', 'year-naics-occ_code', 'year-naics-occ_code.1','year.2', 'naics.2', 'naics_title.2', 'occ_code.2', 'title_code.1', 'dominant_lifecycle.1', 'occ_title.2', 'occ_title_code.1', 'occ_title_mid.1','group.1', 'naics_codedescription.1', 'keepsy'],axis=1)
# the columns # decrease to 37 from 61
del_duplicated_column.shape

(22026, 37)

## Call OES data by 6-digit occupation code

In [28]:
# try to extract data by occ_code 
dx = del_duplicated_column[del_duplicated_column['occ_code'] == '11-1011']
dx[['year','naics_title', 'tot_emp_num']].head(3)

,year,naics_title,tot_emp_num
2,2007,Residential Building Construction,2700.0
203,2007,Nonresidential Building Construction,3580.0
405,2007,Land Subdivision,550.0


In [0]:
# assign the matrix of "year" and "naics_code" to a specific occ_code, call it 'labour_economy_data'
# first, create a function that generate the lb_econ_matrix for an occ_code
def lb_econ (occ_code,field,data):
    # this function yield a data frame for a given occ_code and a field
    # get all unique naics titles in the data
    feature_list = data['naics_title'].unique()
    # get all unique year in the data
    years = sorted(data['year'].unique())    
    mt_result = pd.DataFrame(0.00, index = years, columns = feature_list)   
    # filter out all rows and columns that have the given occ_code and fields
    df_occ = data[data['occ_code'] == occ_code][['year','naics_title', field]]
    for index, row in df_occ.iterrows():
        try:
            mt_result.loc[row['year']][row['naics_title']] = float(row[field])
        except:
            mt_result.loc[row['year']][row['naics_title']] = np.nan
    return mt_result.round(2)


In [31]:
# An example of how to use the function
data1=del_duplicated_column
# the lb_Econ_features contains all the fields' name that we can choose from, there're 17 all together.
lb_econ_features=['occ_code','tot_emp_num', 'emp_prse', 'pct_total', 'pct_rpt', 'h_mean', 'a_mean', 'mean_prse', 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90', 'a_pct10', 'a_pct25', 'a_median_num', 'a_pct75', 'a_pct90']
#try to input a field name called 'tot_emp_num'
aa=lb_econ ('11-1011','tot_emp_num',data1)
aa.head()

,Residential Building Construction,Nonresidential Building Construction,Land Subdivision,"Foundation, Structure, and Building Exterior Contractors",Building Equipment Contractors,Building Finishing Contractors,Other Specialty Trade Contractors,Lessors of Real Estate,Offices of Real Estate Agents and Brokers,Activities Related to Real Estate,"Architectural, Engineering, and Related Services",Specialized Design Services,Services to Buildings and Dwellings
2007,2700.0,3580.0,550.0,1780.0,5170.0,1420.0,1580.0,930.0,910.0,1470.0,6280.0,710.0,2300.0
2008,2380.0,3900.0,460.0,1780.0,4850.0,1400.0,1530.0,970.0,810.0,1590.0,5800.0,610.0,2220.0
2009,1780.0,3730.0,490.0,1690.0,4750.0,1190.0,1400.0,920.0,690.0,1600.0,5670.0,590.0,2140.0
2010,1320.0,3030.0,410.0,1430.0,3980.0,1010.0,1040.0,1010.0,550.0,1550.0,5050.0,400.0,1700.0
2011,1080.0,2780.0,400.0,1400.0,4150.0,890.0,910.0,990.0,530.0,1410.0,4870.0,420.0,1590.0
